In [ ]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
password = getpass.getpass()

In [ ]:
sk = "sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+sk
engine = create_engine(connection_string)
engine

In [ ]:
connection = engine.connect()

In [ ]:
with engine.connect() as connection:
    ...

In [ ]:
from sqlalchemy import text

with engine.connect() as connection:
    query = text('SELECT * FROM rental')
    result = connection.execute(query)

result

In [ ]:
with engine.connect() as connection:
    query = text('SELECT * FROM rental')
    result = connection.execute(query)
    df = pd.DataFrame(result.all())

df

In [ ]:
def rentals_month(engine, month, year):
    query = f"""
    SELECT * 
    FROM rental 
    WHERE MONTH(rental_date) = {month} 
    AND YEAR(rental_date) = {year};
    """
    rental_data = pd.read_sql_query(query, engine)
    
    return rental_data


In [ ]:
def rental_count_month(rental_data, month, year):
    rental_count = rental_data['customer_id'].value_counts().reset_index()
    rental_count.columns = ['customer_id', f'rentals_{month:02d}_{year}']
    return rental_count

In [ ]:
def compare_rentals(rental_data1, rental_data2):
    merged_rentals = rental_data1.merge(rental_data2, on='customer_id', how='outer')
    merged_rentals = merged_rentals.fillna(0)
    merged_rentals['difference'] = merged_rentals.iloc[:, 1] - merged_rentals.iloc[:, 2]
    return merged_rentals
